**Creating Model Which Use To Make Brochure For Company By Just Providing The Website URL Of Company**

**Importing Libraries**

In [27]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
load_dotenv(override=True)

True

**Declaration of API KEY & Model**

In [28]:
openai = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=os.getenv("Groq_API"))
MODEL="llama-3.3-70b-versatile"

**Featch Different Links From Given Websites and Selects Importent Links For Brochure**

In [29]:
links = fetch_website_links("https://www.volkschem.com/")
links

['company-profile.html',
 'about-us.html',
 'management-team.html',
 'achievement.html',
 'our-products.html',
 'https://www.volkschem.com/our-brand-products.html',
 'https://www.volkschem.com/b2b-product.html',
 'https://www.volkschem.com/institutional.html',
 'https://www.volkschem.com/export-products.html',
 'infastructure.html',
 'manufacturing-unit.html',
 'quality-department.html',
 'images/brochure.pdf',
 'quality-assurance.html',
 'certification.html',
 'ehs-policy.html',
 'product-evolution.html',
 'career.html',
 'contact-us.html',
 'company-overview.html',
 'company-overview.html',
 'vision-and-values.html',
 'ehs-policy.html',
 'management-team.html',
 'certification.html',
 'r-and-d.html',
 'manufacturing.html',
 'our-products.html',
 'https://www.volkschem.com/our-brand-products.html',
 'https://www.volkschem.com/b2b-product.html',
 'https://www.volkschem.com/institutional.html',
 'https://www.volkschem.com/export-products.html',
 'sitemap.html',
 'contact-us.html',
 '#ca

In [30]:
# Removing Unrelevant Links

System_promt_1 = """
You are a link-filtering engine for company brochure generation. You receive a list of URLs or paths from a company website and must return only brochure-relevant landing pages as fully expanded absolute URLs belonging strictly to the base domain
 provided by the user. Always convert every relative path into a full absolute URL using the user’s domain and never invent, replace, modify, or substitute the domain and never output placeholder domains like example.com. Keep only these page
   types if found: home, landing, about, company profile, who we are, team, products, product categories, services, solutions, catalog, contact, support, mission, vision, certifications, awards, achievements, clients, partners, testimonials,
     industries served, leadership, management. Always ignore and remove operational, backend, or technical pages including manufacturing, html, process, facility, production, lab, testing, qc, sops, and also remove mailto, tel, whatsapp,
       javascript, anchors, files, downloads, non-http protocols, external domains, social links, login, admin, and anything not belonging to the user’s base domain. Output only a JSON list named 'links' where each item contains a 'type' and 
       the correct absolute 'url'. Do not add explanations or commentary unless asked. Only return clean filtered links in the required JSON format.

You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}

SYntax:
{
  "links": [
    {
      "type": "string",
      "url": "string"
    }
  ]
}

"""

def get_links_user_prompt(url):
    user_prompt = f"""
I extracted links from a company website.
Filter the links and return only brochure-relevant pages such as About, Products, Services, Catalog, Contact, Mission, Vision, Company Profile, Certifications, Clients, etc.
Remove all irrelevant, technical, operational, policy, media, or login/signup pages.
Also remove anything like manufacturing.html, process, facility, or deep backend pages.

Most Important:
Every link you return must be a full absolute URL, NOT a relative path.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [31]:
print(get_links_user_prompt("https://www.volkschem.com/"))


I extracted links from a company website.
Filter the links and return only brochure-relevant pages such as About, Products, Services, Catalog, Contact, Mission, Vision, Company Profile, Certifications, Clients, etc.
Remove all irrelevant, technical, operational, policy, media, or login/signup pages.
Also remove anything like manufacturing.html, process, facility, or deep backend pages.

Most Important:
Every link you return must be a full absolute URL, NOT a relative path.

Links (some might be relative links):

company-profile.html
about-us.html
management-team.html
achievement.html
our-products.html
https://www.volkschem.com/our-brand-products.html
https://www.volkschem.com/b2b-product.html
https://www.volkschem.com/institutional.html
https://www.volkschem.com/export-products.html
infastructure.html
manufacturing-unit.html
quality-department.html
images/brochure.pdf
quality-assurance.html
certification.html
ehs-policy.html
product-evolution.html
career.html
contact-us.html
company-o

In [32]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": System_promt_1},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [33]:
select_relevant_links("https://www.volkschem.com/")

Selecting relevant links for https://www.volkschem.com/ by calling llama-3.3-70b-versatile
Found 15 relevant links


{'links': [{'type': 'About page',
   'url': 'https://www.volkschem.com/about-us.html'},
  {'type': 'Company Profile page',
   'url': 'https://www.volkschem.com/company-profile.html'},
  {'type': 'Management page',
   'url': 'https://www.volkschem.com/management-team.html'},
  {'type': 'Achievement page',
   'url': 'https://www.volkschem.com/achievement.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/our-products.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/our-brand-products.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/b2b-product.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/institutional.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/export-products.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/bio-pesticide.html'},
  {'type': 'Products page',
   'url': 'https://www.volkschem.com/bio-fertilizer.html'},
  {'type': 'Certificati

**Creating Brochure**

In [ ]:
# Fetching Page Contents for Landing Page and Relevant Links

def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [37]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
create_brochure("Volkschem", "https://www.volkschem.com/")

Selecting relevant links for https://www.volkschem.com/ by calling llama-3.3-70b-versatile
Found 16 relevant links


# Introduction to Volkschem Crop Science (P) Limited
Volkschem Crop Science (P) Limited is a leading manufacturer of bio pesticides, bio larvicides, and bio fertilizers, dedicated to providing effective agricultural products to resolve farming-related issues. Established in 2011, the company has grown to become one of the top bio pesticide manufacturers in Ahmedabad, Gujarat.

## Company Overview
Volkschem Crop Science (P) Limited has a strong infrastructure, with separate facilities for processing, packaging, storage, warehousing, and distribution of its products. The company's state-of-the-art infrastructure is equipped with modern equipment, machines, and technology, enabling it to surpass client expectations. With a team of highly professional and experienced individuals, Volkschem Crop Science (P) Limited has gained recognition in both national and international markets.

## Products and Services
The company offers a wide range of products, including:
* Plant Growth Regulator
* Larvicide
* Plant Growth Promoter
* Bio Fertilizers
* Insecticides
* Herbicides
* Fungicides
* Mix Micronutrient

Volkschem Crop Science (P) Limited is committed to delivering high-quality products that meet industry standards, ensuring customer satisfaction and value for money.

## Customers
The company's products cater to the agricultural sector, benefiting farmers and end-users. With a strong focus on customer satisfaction, Volkschem Crop Science (P) Limited has built a loyal customer base in India and abroad.

## Careers
Volkschem Crop Science (P) Limited offers career opportunities for individuals who share the company's vision and values. The company's dedicated team is made up of experts in their respective fields, and it is committed to providing a supportive and growth-oriented work environment.

## Culture
At Volkschem Crop Science (P) Limited, the culture is centered around innovation, quality, and customer satisfaction. The company's core values include:
* Ensuring responsible and limitless growth
* Applying innovative approaches to its products and processes
* Commitment to environmental, health, and safety (EHS) policies

By joining Volkschem Crop Science (P) Limited, individuals can be part of a dynamic and growth-oriented organization that prioritizes employee development and well-being.

## Contact Us
For more information about Volkschem Crop Science (P) Limited, its products, and career opportunities, please visit the company's website or contact us directly.